# 1. Préambule

## 1.1 Problématique

La très jeune start-up de l'AgriTech, nommée "**Fruits**!", cherche à proposer des solutions innovantes pour la récolte des fruits.
La volonté de l’entreprise est de préserver la biodiversité des fruits en permettant des traitements spécifiques pour chaque espèce de fruits en développant des robots cueilleurs intelligents.
La start-up souhaite dans un premier temps se faire connaître en mettant à disposition du grand public une application mobile qui permettrait aux utilisateurs de prendre en photo un fruit et d'obtenir des informations sur ce fruit.
Pour la start-up, cette application permettrait de sensibiliser le grand public à la biodiversité des fruits et de mettre en place une première version du moteur de classification des images de fruits.
De plus, le développement de l’application mobile permettra de construire une première version de l'architecture **Big Data** nécessaire.

## 1.2 Objectifs dans ce projet

1. Développer une première chaîne de traitement des données qui comprendra le **preprocessing** et une étape de **réduction de dimension**.
2. Tenir compte du fait que <u>le volume de données va augmenter très rapidement</u> après la livraison de ce projet, ce qui implique de:
 - Déployer le traitement des données dans un environnement **Big Data**
 - Développer les scripts en **pyspark** pour effectuer du **calcul distribué**

## 1.3 Déroulement des étapes du projet

Le projet va être réalisé en 2 temps, dans deux environnements différents. Nous allons dans un premier temps développer et exécuter notre code en local, en travaillant sur un nombre limité d'images à traiter.
Une fois les choix techniques validés, nous déploierons notre solution dans un environnement Big Data en mode distribué.

<u>Pour cette raison, ce projet sera divisé en 3 parties<u>:
- Liste des choix techniques généraux retenus
- Déploiement de la solution en local
- Déploiement de la solution dans le cloud

# 2. Choix techniques généraux retenus

## 2.1 Calcul distribué

L’énoncé du projet nous impose de développer des scripts en **pyspark** afin de prendre en compte l’augmentation très rapide du volume de donné après la livraison du projet.

Lorsque l’on parle de traitement de bases de données sur python, on pense immédiatement à la librairie pandas. Cependant, lorsqu’on a affaire à des bases de données trop massives, les calculs deviennent trop lents. Heureusement, il existe une autre librairie python, assez proche de pandas, qui permet de traiter des très grandes quantités de données : PySpark. Apache Spark est un framework open-source développé par l’AMPLab de UC Berkeley permettant de traiter des bases de données massives en utilisant le calcul distribué, technique qui consiste à exploiter plusieurs unités de calcul réparties en clusters au profit d’un seul projet afin de diviser le temps d’exécution d’une requête. Spark a été développé en Scala et est au meilleur de ses capacités dans son langage natif. Cependant, la librairie PySpark propose de l’utiliser avec le langage Python, en gardant des performances similaires à des implémentations en Scala. Pyspark est donc une bonne alternative à la librairie pandas lorsqu’on cherche à traiter des jeux de données trop volumineux qui entraînent des calculs trop chronophages.

Comme nous le constatons, **pySpark** est un moyen de communiquer avec **Spark** via le langage **Python**. **Spark**, quant à lui, est un outil qui permet de gérer et de coordonner l'exécution de tâches sur des données à travers un groupe d'ordinateurs. <u>Spark (ou Apache Spark) est un framework open source de calcul distribué in-memory pour le traitement et l'analyse de données massives<u>.

Les applications Spark se composent d’un pilote (« driver process ») et de plusieurs exécuteurs (« executor processes »). Il peut être configuré pour être lui-même l’exécuteur (local mode) ou en utiliser autant que nécessaire pour traiter l’application, Spark prenant en charge la mise à l’échelle automatique par une configuration d’un nombre minimum et maximum d’exécuteurs.

Le driver (parfois appelé « Spark Session ») distribue et planifie les tâches entre les différents exécuteurs qui les exécutent et permettent un traitement réparti. Il est le responsable de l’exécution du code sur les différentes machines.
Chaque exécuteur est un processus Java Virtual Machine (JVM) distinct dont il est possible de configurer le nombre de CPU et la quantité de mémoire qui lui est alloué. Une seule tâche peut traiter un fractionnement de données à la fois.
Nous utiliserons donc **Spark** et nous l’exploiterons à travers des scripts python grâce à **PySpark**, et nous **réaliserons les opérations sur un cluster de machine**.

## 2.2 Transfert Learning

L'énoncé du projet nous demande également de réaliser une première chaîne de traitement des données qui comprendra le preprocessing et une étape de réduction de dimension.
Il est également précisé qu'il n'est pas nécessaire d'entraîner un modèle pour le moment.
Nous décidons de partir sur une solution de **transfert learning**.
Simplement, le **transfert learning** consiste à utiliser la connaissance déjà acquise par un modèle entraîné (ici **MobileNetV2**) pour l'adapter à notre problématique.
Nous allons fournir au modèle nos images, et nous allons </u>récupérer l'avant dernière couche</u> du modèle. En effet la dernière couche de modèle est une couche softmax qui permet la classification des images ce que nous ne souhaitons pas dans ce projet.
L'avant dernière couche correspond à un **vecteur réduit** de dimension (1,1,1280).
Cela permettra de réaliser une première version du moteur pour la classification des images des fruits.
**MobileNetV2** a été retenu pour sa </u>rapidité d'exécution</u>, particulièrement adaptée pour le traitement d'un gros volume de données ainsi que la </u>faible dimensionnalité du vecteur de caractéristique en sortie</u> (1,1,1280)

# 3. Execution sur un cluster EMR  

<u>Avant de commencer</u>, il faut s'assurer d'utiliser le **kernel pyspark**.
En utilisant ce kernel, une session spark est créé à l'exécution de la première cellule**.

## 3.1 Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1720979201060_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1720979201060_0002,pyspark,idle,Link,Link,None,✔


## 3.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

## 3.3 Import des librairies

In [3]:
import io
import os
import numpy as np
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [4]:
PATH = 's3://alex-martineau-p9-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://projet9-oba-data
PATH_Data:   s3://projet9-oba-data/Test
PATH_Result: s3://projet9-oba-data/Results

## 3.5 Traitement des données

<u>Dans la suite de notre flux de travail, nous allons successivement</u> :
1. Préparer nos données
    1. Importer les images dans un dataframe **pandas UDF**
    2. Associer aux images leur **label**
    3. Préprocesser en **redimensionnant nos images pour qu'elles soient compatibles avec notre modèle**
2. Préparer notre modèle
    1. Importer le modèle **MobileNetV2**
    2. Créer un **nouveau modèle** dépourvu de la dernière couche de MobileNetV2
3. Définir le processus de chargement des images et l'application de leur featurisation à travers l'utilisation de pandas UDF
3. Exécuter les actions d'extraction de features
4. Enregistrer le résultat de nos actions
5. Tester le bon fonctionnement en chargeant les données enregistrées

### a) Chargement des données

Les images sont chargées au format binaire, ce qui offre, plus de souplesse dans la façon de prétraiter les images.
Avant de charger les images, nous spécifions que nous voulons charger uniquement les fichiers dont l'extension est **jpg**.
Nous indiquons également de charger tous les objets possibles contenus dans les sous-dossiers du dossier communiqué.

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

In [5]:
images = spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg").option("recursiveFileLookup", "true").load(PATH_Data)

images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Je ne conserve que le **path** de l'image et j'ajoute une colonne contenant les **labels** de chaque image</u> :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------------+----------+
|path                                               |label     |
+---------------------------------------------------+----------+
|s3://projet9-oba-data/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://projet9-oba-data/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://projet9-oba-data/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://projet9-oba-data/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://projet9-oba-data/Test/Watermelon/r_95_100.jpg |Watermelon|
+---------------------------------------------------+----------+
only showing top 5 rows

None

### b) Préparation du modèle

Je vais utiliser la technique du **transfert learning** pour extraire les features des images. J'ai choisi d'utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée à d'autres modèles comme *VGG16* par exemple.

Il existe une dernière couche qui sert à classer les images selon 1000 catégories que nous ne voulons pas utiliser. L'idée dans ce projet est de récupérer le **vecteur de caractéristiques de dimensions (1,1,1280)** qui servira, plus tard, au travers d'un moteur de classification à reconnaitre les différents fruits du jeu de données.
Comme d'autres modèles similaires, **MobileNetV2**, lorsqu'on l'utilise en incluant toutes ses couches, attend obligatoirement des images de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [8]:
# Création du modèle MobileNetV2 avec l'ensemble des couches :
model = MobileNetV2(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Affichage du résumé de notre nouveau modèle où nous constatons que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [9]:
# Création du modèle spécifique (retrait de la dernière couche): 
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

# Diffusion des poids du modèle :
brodcast_weights = sc.broadcast(new_model.get_weights())

# Résumé du modèle
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser ensuite les poids aux différents workeurs.

<u>Mettons cela sous forme de fonction</u>.

### c) Fonctions de chargement & de traitement des images & creation des features

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.
L'empilement des appels est la suivante :
- Pandas UDF
- featuriser une série d'images pd.Series
- prétraiter une image

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### d) Exécution des actions d'extraction de features

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), peuvent rencontrer des erreurs de type Out Of Memory (OOM).
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'
Je n'utiliserai pas cette commande dans ce projet et je laisse donc la commande en commentaire.

Nous pouvons maintenant exécuter la featurisation sur l'ensemble de notre DataFrame Spark.
<u>REMARQUE</u> : Cela peut prendre beaucoup de temps, tout dépend du volume de données à traiter.
Notre jeu de données de **Test** contient **22819 images**.

In [16]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

# Extraction des features en utilisant 21 partitions : 
features_df = images.repartition(21).select(col("path"), col("label"), featurize_udf("content").alias("features"))

features_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### e) ACP

Cette ACP est réalisée dans Spark et non pas dans scikit-learn.

In [19]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.feature import PCA

# Définir une UDF pour convertir un array de floats en un vecteur
array_to_vector = udf(lambda x: Vectors.dense(x), VectorUDT())

# Appliquer l'UDF à la colonne `features`
features_df = features_df.withColumn("features_vector", array_to_vector(features_df["features"]))

# Utiliser cette nouvelle colonne avec PCA
pca = PCA(k=200, inputCol="features_vector", outputCol="reduced_features")
pca_model = pca.fit(features_df)

# Examiner la variance expliquée pour ajuster 'k'
print("Variance expliquée par chaque composante principale :")
print(pca_model.explainedVariance)

# Somme de la variance expliquée par les k composantes principales
total_variance_explained = pca_model.explainedVariance.sum()
print("Somme de la variance expliquée par les k composantes principales :", total_variance_explained)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Variance expliqu?e par chaque composante principale :
[0.1014099130425472,0.08005981651535385,0.06350302718082,0.05014785531069757,0.035354277761087655,0.0291599319836993,0.027732146814681394,0.02284889389541205,0.01985943570194587,0.0190915273951655,0.016537073756515547,0.014662889455521197,0.013981834223290643,0.01368578231395087,0.013347894522772534,0.012500332847094574,0.011550825659275506,0.010749219889392,0.009800369937620823,0.009710070622970876,0.009158305487468892,0.008282988110141443,0.007887717765771638,0.007507839166401488,0.007148164607314943,0.0070723543600667215,0.00683641148804439,0.006218355823197223,0.006122050457134876,0.005890478930769407,0.0057031458892763546,0.005575825947177239,0.005273852071953272,0.005050017949146899,0.004785237405823075,0.0046883810733760085,0.004595275012393054,0.004346392145729127,0.004235182372862692,0.004092068423945076,0.003902364777139975,0.0038892783689277316,0.003844463960579333,0.003723693952167754,0.0036299162765013842,0.003422199933

In [20]:
# Transformer le DataFrame pour obtenir les caractéristiques réduites
features_reduced_df = pca_model.transform(features_df)

# Sélectionner les colonnes d'intérêt et afficher le résultat
final_df = features_reduced_df.select("path", "label", "reduced_features")
final_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|                path|         label|    reduced_features|
+--------------------+--------------+--------------------+
|s3://projet9-oba-...|    Watermelon|[-3.1191640377446...|
|s3://projet9-oba-...|    Watermelon|[1.10923363586790...|
|s3://projet9-oba-...|    Watermelon|[-1.9238940051017...|
|s3://projet9-oba-...|    Watermelon|[-3.0128143651585...|
|s3://projet9-oba-...|Pineapple Mini|[-6.4304468478088...|
+--------------------+--------------+--------------------+
only showing top 5 rows

In [21]:
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- reduced_features: vector (nullable = true)

In [22]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import VectorUDT

# Définir une UDF pour convertir un vecteur en array de floats
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

# Appliquer l'UDF à la colonne `reduced_features` pour créer une nouvelle colonne `features`
final_df = final_df.withColumn("features", vector_to_array(final_df["reduced_features"]))

# Supprimer la colonne `reduced_features` si elle n'est plus nécessaire
final_df = final_df.drop("reduced_features")

# Vérifier le schéma du DataFrame modifié
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

# 4. Enregistrement du résultat

<u>Rappel du PATH où seront inscrits les fichiers au format "**parquet**" contenant nos résultats, à savoir, un DataFrame contenant 3 colonnes</u> :
 1. Path des images
 2. Label de l'image
 3. Vecteur de caractéristiques de l'image

<u>Enregistrement des données traitées au format "**parquet**"</u> :

In [23]:
print(PATH_Result)

final_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://projet9-oba-data/Results

# 5. Chargement des données enregistrées et validation du résultat

<u>On charge les données fraichement enregistrées dans un **DataFrame Pandas**, puis on affiche les 5 premières lignes du DataFrame</u> :

In [25]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280</u> :

In [27]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(200,)

In [28]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22688, 3)

<u>On peut également constater la présence des fichiers au format "**parquet**" sur le **serveur S3**</u>

# 6. Sauvegarde des résultats en csv

In [29]:
# Convertir la colonne 'features' contenant des listes en un DataFrame séparé
features_df = pd.DataFrame(df['features'].tolist())

# Renommer les colonnes du nouveau DataFrame pour refléter qu'elles sont des composantes
features_df.columns = [f'feature_{i}' for i in range(features_df.shape[1])]

# Concaténer le DataFrame original (sans la colonne 'features') avec le nouveau DataFrame des composantes
df_final = pd.concat([df.drop('features', axis=1), features_df], axis=1)

# Enregistrement du DataFrame en tant que fichier CSV sur S3
df_final.to_csv(PATH_Result + '/df_results_aws_emr.csv', index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Vérification de l'enregistrement : 
df_final = pd.read_csv(PATH_Result + '/df_results_aws_emr.csv')

# Affichage des 5 premières lignes : 
df_final.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ... feature_199
0  s3://projet9-oba-data/Test/Watermelon/r_78_100...  ...    0.600454
1  s3://projet9-oba-data/Test/Watermelon/r_169_10...  ...   -0.278663
2  s3://projet9-oba-data/Test/Watermelon/202_100.jpg  ...   -0.325102
3   s3://projet9-oba-data/Test/Raspberry/238_100.jpg  ...    0.748535
4   s3://projet9-oba-data/Test/Raspberry/235_100.jpg  ...    0.619722

[5 rows x 202 columns]

# 7. Conclusion

Nous avons réalisé ce projet en tenant compte des contraintes qui nous ont été imposées.

Nous avons fait le choix de réaliser du **transfert learning** à partir du model **MobileNetV2**. Ce modèle a été retenu pour sa **légèreté** et sa **rapidité d'exécution** ainsi que pour la **faible dimension de son vecteur en sortie**.
Les résultats ont été enregistrés sur disque en plusieurs partitions au format "**parquet**".

Dans ce notebook, nous créée un **réel cluster de calculs**. L'objectif était de pouvoir **anticiper une future augmentation de la charge de travail**.
Le meilleur choix retenu a été l'utilisation du prestataire de services **Amazon Web Services** qui nous permet de **louer à la demande de la puissance de calculs**, pour un **coût tout à fait acceptable**. Ce service se nomme **EC2** et se classe parmi les offres **Infrastructure As A Service** (IAAS).

Nous sommes allés plus loin en utilisant un service de plus haut niveau (**Plateforme As A Service** PAAS) en utilisant le service **EMR** qui nous permet d'un seul coup d'**instancier plusieurs serveur (un cluster)** sur lesquels nous avons pu demander l'installation et la configuration de plusieurs programmes et librairies nécessaires à notre projet comme **Spark**, **Hadoop**, **JupyterHub** ainsi que la librairie **TensorFlow**.
En plus d'être plus **rapide et efficace à mettre en place**, nous avons la **certitude du bon fonctionnement de la solution**, celle-ci ayant été préalablement validé par les ingénieurs d'Amazon.
Nous avons également pu installer, sans difficulté, **les packages nécessaires sur l'ensembles des machines du cluster**.
Enfin, avec très peu de modification, et plus simplement encore, nous avons pu **exécuter notre notebook comme nous l'avions fait localement**. Nous avons cette fois-ci exécuté le traitement sur **l'ensemble des images de notre dossier "Test"**.

Nous avons opté pour le service **Amazon S3** pour **stocker les données de notre projet**. S3 offre, pour un faible coût, toutes les conditions dont nous avons besoin pour stocker et exploiter de manière efficace nos données. L'espace alloué est potentiellement **illimité**, mais les coûts seront fonction de l'espace utilisé.

Il nous sera **facile de faire face à une montée de la charge de travail** en **redimensionnant** simplement notre cluster de machines (horizontalement et/ou verticalement au besoin), les coûts augmenteront en conséquence mais resteront nettement inférieurs aux coûts engendrés par l'achat de matériels ou par la location de serveurs dédiés.